---
title: "Project Code"
subtitle: Team name
author: Author 1, Author 2, Author 3, and Author 4 
date: 02/27/2023
number-sections: true
abstract: _This file contains the code for the project on <>, as part of the STAT303-2 course in Winter 2023_.
format: 
  html:
    toc: true
    toc-title: Contents
    self-contained: true
    font-size: 100%
    toc-depth: 4
    mainfont: serif
jupyter: python3
---

## Length of the code {-}
No restriction

**Delete this section from the report, when using this template.** 

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
import math
import itertools
import time

## Data quality check / cleaning / preparation 

Put code with comments. The comments should explain the code such that it can be easily understood. You may put text *(in a markdown cell)* before a large chunk of code to explain the overall purpose of the code, if it is not intuitive. **Put the name of the person / persons who contributed to each code chunk / set of code chunks.** An example is given below.

### Data quality check
*By Elton John*

The code below visualizes the distribution of all the variables in the dataset, and their association with the response.

### Data cleaning
*By NAMES*

From the data quality check we realized that:

1. Some of the columns that should have contained only numeric values, specifically <>, <>, and <> have special characters such as \*, #, %. We'll remove these characters, and convert the datatype of these columns to numeric.

2. Some of the columns have more than 60% missing values, and it is very difficult to impute their values, as the values seem to be missing at random with negligible association with other predictors. We'll remove such columns from the data.

3. The column `number_of_bedrooms` has some unreasonably high values such as 15. As our data consist of single-family homes in Evanston, we suspect that any value greater than 5 may be incorrect. We'll replace all values that are greater than 5 with an estimate obtained using the $K$-nearest neighbor approach.

4. The columns `house_price` has some unreasonably high values. We'll tag all values greater than 1 billion dollars as "potentially incorrect observation", to see if they distort our prediction / inference later on.

The code below implements the above cleaning.

#...Code with comments...#

### Data preparation
*By Kegan Grace and Albert Wang*

Our dataset was a collection of csv files for each decade from 1950-2019, so we needed to concatenate all the files together to expand our dataset and number of observations.

y1950 = pd.read_csv('1950.xls')
y1960 = pd.read_csv('1960.xls')
y1970 = pd.read_csv('1970.xls')
y1980 = pd.read_csv('1980.xls')
y1990 = pd.read_csv('1990.xls')
y2000 = pd.read_csv('2000.xls')
y2010 = pd.read_csv('2010.xls')
all_data = pd.concat([y1950,y1960,y1970,y1980,y1990,y2000,y2010])

from sklearn.model_selection import train_test_split
X = all_data.drop('pop', axis=1)
y = all_data['pop']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

## Exploratory data analysis

Put code with comments. The comments should explain the code such that it can be easily understood. You may put text *(in a markdown cell)* before a large chunk of code to explain the overall purpose of the code, if it is not intuitive. **Put the name of the person / persons who contributed to each code chunk / set of code chunks.**

### Histogram of Popularity Values
*By Albert Wang*

#histogram/dist of song popularity values
sns.displot(df['pop'], bins=30)
plt.show()

### Numerical variables against song popularity
*By Grant Li*

#visualizations of all numerical variables (including non-audial features) agaisnt song popularity
predictors = all_data.corr().columns
fig, axes = plt.subplots(4,3,figsize=(20,20))
p = 0
for i in range(4):
    for j in range(3):
        
        if p >= 12:
            break
        predictor = predictors[p]
        
        sns.scatterplot(ax=axes[i,j], x=predictor,y='pop',data=all_data)
        p+=1
plt.show()

## Developing the model

Put code with comments. The comments should explain the code such that it can be easily understood. You may put text *(in a markdown cell)* before a large chunk of code to explain the overall purpose of the code, if it is not intuitive. **Put the name of the person / persons who contributed to each code chunk / set of code chunks.**

### Checking Multicollinearity
*By Grant Li*

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

X = train[['dB', 'dnce', 'nrgy', 'acous', 'dur', 'val', 'spch', 'live']]

X = add_constant(X)
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns

for i in range(len(X.columns)):
    vif_data.loc[i,'VIF'] = variance_inflation_factor(X.values, i)

print(vif_data)

### Model Testing
*By Nate Kim*

# initial model
model = smf.ols(formula='pop~acous+dB+dur+dnce+nrgy+val', data=train).fit()
print(model.summary())
#test MAE
pred_pop = model.predict(test)
mae = (np.abs(test['pop'] - pred_pop)).mean()
print("Test MAE is", mae)
#train MAE
mae_train = (np.abs(train['pop'] - pred_pop)).mean()
print("Train MAE is",mae_train)

#model with **2 transformation on duration
model = smf.ols(formula='pop~acous+dB+dur+dnce+nrgy+I(dur**2)', data=train).fit()
print(model.summary())
#test MAE
pred_pop = model.predict(test)
mae = (np.abs(test['pop'] - pred_pop)).mean()
print("Test MAE is", mae)
#train MAE
mae_train = (np.abs(train['pop'] - pred_pop)).mean()
print("Train MAE is",mae_train)

#model with log transformation on duration
model = smf.ols(formula='pop~acous+dB+dur+dnce+nrgy+I(np.log(dur))', data=train).fit()
print(model.summary())
#test MAE
pred_pop = model.predict(test)
mae = (np.abs(test['pop'] - pred_pop)).mean()
print("Test MAE is", mae)
#train MAE
mae_train = (np.abs(train['pop'] - pred_pop)).mean()
print("Train MAE is",mae_train)

# model with sqrt tranformation on dur
model = smf.ols(formula='pop~acous+dB+dur+dnce+nrgy+I(np.sqrt(dur))', data=train).fit()
print(model.summary())
#test MAE
pred_pop = model.predict(test)
mae = (np.abs(test['pop'] - pred_pop)).mean()
print("Test MAE is", mae)
#train MAE
mae_train = (np.abs(train['pop'] - pred_pop)).mean()
print("Train MAE is",mae_train)



### Code fitting the final model

model_best = smf.ols(formula='pop~acous+dB+dur+nrgy*dnce+val+I(np.log(dur))', data=train).fit()
print(model_best.summary())
#test MAE
pred_pop = model_best.predict(test)
mae = (np.abs(test['pop'] - pred_pop)).mean()
print("Test MAE is", mae)
#train MAE
mae_train = (np.abs(train['pop'] - pred_pop)).mean()
print("Train MAE is",mae_train)

Put the code(s) that fit the final model(s) in separate cell(s), i.e., the code with the `.ols()` or `.logit()` functions.

## Conclusions and Recommendations to stakeholder(s)

You may or may not have code to put in this section. Delete this section if it is irrelevant.

In [2]:
#outliers
sns.scatterplot(x=model_best.fittedvalues, y=model_best.resid, color='orange')
sns.lineplot(x=[pred_pop.min(),pred_pop.max()], y=[0,0],color='blue')
plt.xlabel('Predicted popularity')
plt.ylabel('Residual')
plt.show()

NameError: name 'sns' is not defined

In [3]:
out = model_best.outlier_test()
# returns a df w the first col as studentized residuals
# res plot, this time w studentized residuals
sns.scatterplot(x=model_best.fittedvalues, y = out.student_resid, color='orange')
sns.lineplot(x = [model_best.fittedvalues.min(),model_best.fittedvalues.max()],y = [0,0],color = 'blue')
plt.xlabel('Predicted popularity')
plt.ylabel('Studentized residuals')
plt.show()

NameError: name 'model_best' is not defined

In [4]:
print(np.sum(np.abs(out.student_resid) > 3)) # 3 outliers

NameError: name 'np' is not defined

In [5]:
# leverage calculations
influence = model_best.get_influence()
leverage = influence.hat_matrix_diag
cutoff = 4*(model_best.df_model+1)/model_best.nobs
print(np.sum(leverage>cutoff)) #6 high leverage points

NameError: name 'model_best' is not defined

In [6]:
#Dropping influential points from data
train_filtered = train.drop(np.intersect1d(np.where(np.abs(out.student_resid)>3)[0],
                                           (np.where(leverage>cutoff)[0])))
train_filtered.shape[0]-train.shape[0]
#no influential points

NameError: name 'train' is not defined